<h2>Segmenting and Clustering Neighbourhoods in Toronto</h2>

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. 
For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.
1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.
3. To create the above dataframe:
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
4. Submit a link to your Notebook on your Github repository. (10 marks)

<h3>All the 3 tasks of <i>web scraping</i>, <i>cleaning</i> and <i>clustering</i> are implemented in the same notebook for the ease of evaluation.</h3>

<h3>Installing and Importing the required Libraries</h3>

In [67]:
#install the BeautifulSoup package for web scraping
!pip install beautifulsoup4
from bs4 import BeautifulSoup

#install lxml for processing XML and HTML
!pip install lxml

#import library for requests handling
import requests as rq
#import library for data analsysis
import pandas as pd
#import library for vectorized data handling
import numpy as np

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


<h3>Task 1 Scraping the Wikipedia page for the table of postal codes of Canada and create a dataframe</h3>

BeautifulSoup Library of Python is used for scraping of data table from the Wikipedia page, and title of the webpage is printed to checkas well as head of the table of postal codes of Canada is printed.

In [68]:
#scrapping the wiki page from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
wiki_source = rq.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
wiki_soup = BeautifulSoup(wiki_source,'lxml')
print(wiki_soup.title)

<title>List of postal codes of Canada: M - Wikipedia</title>


<h3>Cover html table to Pandas DataFrame for cleaning and preprocessing.</h3>

In [69]:
#retrieve table
wiki_tab = str(wiki_soup.table)
dfs = pd.read_html(wiki_tab)

#convert table to dataframe
df=dfs[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h3>Table data preprocessing and cleaning</h3>

In [74]:
#change column names to column_names = ['Postalcode','Borough','Neighborhood']
df=df.rename(columns={df.columns[0]:'Postalcode',df.columns[1]:'Borough',df.columns[2]:'Neighborhood'})

# Drop rows where Borough is 'Not assigned' as required to oinly process the cells that have an assigned borough and ignore cells with a borough that is Not assigned.
df_noborough = df[df.Borough != 'Not assigned']

# Merge neighbourhoods with same Postalcode
df_merged = df_noborough.groupby(['Postalcode','Borough'], sort=False)['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df_merged = pd.DataFrame(df_merged)
df_merged = df_merged.rename(columns={df.columns[0]:'Postalcode',df.columns[1]:'Borough',df.columns[2]:'Neighborhood'})
df_merged.reset_index(inplace=True)
df_merged

# Replace neighbourhoods which are 'Not assigned' with names of Borough
df_merged['Neighborhood'] = np.where(df_merged['Neighborhood'] == 'Not assigned',df_merged['Borough'], df_merged['Neighborhood'])
df_merged.head(12)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [75]:
# Shape of data frame
df_merged.shape

(103, 3)